# Python. Integracja danych z różnych źródeł.

___Marcin Koźniewski___


Dzisiaj zgłębimy metody w pandas umożliwiające manipulacje danymi. Pandas inspirowane było strukturą danych data.frame z języka R. R jest natomiast językiem funkcyjnym, co ma swoje konsekwencje w implementacji klasy DataFrame.


Stwórzmy najpierw ramkę danych, którą będziemy się bawić.

In [1]:
import pandas as pd
import numpy as np
rng = np.random.RandomState(314) #ustalamy ziarno aby mieć odtwarzalne wyniki

df = pd.DataFrame(
    {
        'binarne': [1,1,1,0,0,0],
        'key': ['A', 'B', 'C', 'A', 'B', 'C'],
        'data': range(6),
        'losowe': rng.rand(6), # 6 wartości losowych
        'losowe2': rng.rand(6),
        'losowe3': rng.rand(6)
    },
    columns=['binarne','key', 'data','losowe','losowe2','losowe3']
)
df.head(6) #df #df.head()

,binarne,key,data,losowe,losowe2,losowe3
0,1,A,0,0.916874,0.727951,0.122291
1,1,B,1,0.588542,0.260480,0.386006
2,1,C,2,0.265048,0.911763,0.840081
3,0,A,3,0.783205,0.260757,0.278179
4,0,B,4,0.918001,0.766376,0.069914
5,0,C,5,0.827355,0.261531,0.633110


Wypisanie podstawowych statystyka opisowych

In [2]:
df.describe()

,binarne,data,losowe,losowe2,losowe3
count,6.000000,6.000000,6.000000,6.000000,6.000000
mean,0.500000,2.500000,0.716504,0.531477,0.388263
std,0.547723,1.870829,0.252061,0.302654,0.299433
min,0.000000,0.000000,0.265048,0.260480,0.069914
25%,0.000000,1.250000,0.637208,0.260950,0.161263
50%,0.500000,2.500000,0.805280,0.494741,0.332092
75%,1.000000,3.750000,0.894494,0.756770,0.571334
max,1.000000,5.000000,0.918001,0.911763,0.840081


Czasem potrzeba jest zamienić wartości zmiennych. Poniżej zamiana wartości całkowitych na ciągi znaków

In [3]:
mapToGroup = {0 : 'val0', 
              1 : 'val1'}
df["binarne_str"] = df['binarne'].map(mapToGroup)
df

,binarne,key,data,losowe,losowe2,losowe3,binarne_str
0,1,A,0,0.916874,0.727951,0.122291,val1
1,1,B,1,0.588542,0.260480,0.386006,val1
2,1,C,2,0.265048,0.911763,0.840081,val1
3,0,A,3,0.783205,0.260757,0.278179,val0
4,0,B,4,0.918001,0.766376,0.069914,val0
5,0,C,5,0.827355,0.261531,0.633110,val0


Dyskretyzacja

In [4]:
df['losoweDysk1'] = pd.cut(df['losowe'],
                           [0, 0.5, 0.75, 1], 
                           labels = ['below', 'high', 'veryhigh'])
df

,binarne,key,data,losowe,losowe2,losowe3,binarne_str,losoweDysk1
0,1,A,0,0.916874,0.727951,0.122291,val1,veryhigh
1,1,B,1,0.588542,0.260480,0.386006,val1,high
2,1,C,2,0.265048,0.911763,0.840081,val1,below
3,0,A,3,0.783205,0.260757,0.278179,val0,veryhigh
4,0,B,4,0.918001,0.766376,0.069914,val0,veryhigh
5,0,C,5,0.827355,0.261531,0.633110,val0,veryhigh


In [5]:
#Dyskretyzacja na kwantylach:
pd.qcut(df['losowe2'],
        [0.0, 0.25, 0.75, 1.0], 
        labels = ['low','medium','high'])
#df[column]=pd.qcut(df[column].astype('float', errors='ignore'),[0.0,0.25,0.75,1.0], labels=['low','medium','high'])

0    medium
1       low
2      high
3       low
4      high
5    medium
Name: losowe2, dtype: category
Categories (3, object): ['low' < 'medium' < 'high']

Wypisanie liczebności występowania wartości w zadanej kolumnie

In [6]:
df["losoweDysk1"].value_counts()

losoweDysk1
veryhigh    4
below       1
high        1
Name: count, dtype: int64

Proszę zwrócić uwagę na typy danych wynikające z różnych działań. np. cut/qcut -> category

In [7]:
df.dtypes


binarne           int64
key              object
data              int64
losowe          float64
losowe2         float64
losowe3         float64
binarne_str      object
losoweDysk1    category
dtype: object

Wiecie już, że mamy dostęp do ciągów znaków i operujemy na jednej kolumnie danych (zmiennej) jak byśmy operowali na jednym ciągu znaków.

In [8]:
df["binarne_str"].str[3:]

0    1
1    1
2    1
3    0
4    0
5    0
Name: binarne_str, dtype: object

In [9]:
df["binarne_str"].str[3:].astype('int', errors='ignore')

0    1
1    1
2    1
3    0
4    0
5    0
Name: binarne_str, dtype: int32

In [10]:
df["binarne_str"].str.upper()

0    VAL1
1    VAL1
2    VAL1
3    VAL0
4    VAL0
5    VAL0
Name: binarne_str, dtype: object

Wydobycie unikalnych wartosci

In [11]:
df["binarne_str"].str.upper().unique()

array(['VAL1', 'VAL0'], dtype=object)

In [12]:
df.losoweDysk1.cat.categories


Index(['below', 'high', 'veryhigh'], dtype='object')

Przypomnijmy, że możemy też odwoływać się do kolumny na zasadzie pola w obiekcie

In [13]:
df.binarne.unique()

array([1, 0], dtype=int64)

In [14]:
df["binarne"].value_counts()

binarne
1    3
0    3
Name: count, dtype: int64

Liczebności poszczególnych kombinacji wartości w formie tabeli

In [15]:
pd.crosstab(df["binarne"], df["key"])

key,A,B,C
binarne,,,
0,1,1,1
1,1,1,1


In [16]:
pd.crosstab(df["binarne"], [df["key"], df["losoweDysk1"]])

key                A    B              C         
losoweDysk1 veryhigh high veryhigh below veryhigh
binarne                                          
0                  1    0        1     0        1
1                  1    1        0     1        0

In [17]:
df.head()

,binarne,key,data,losowe,losowe2,losowe3,binarne_str,losoweDysk1
0,1,A,0,0.916874,0.727951,0.122291,val1,veryhigh
1,1,B,1,0.588542,0.260480,0.386006,val1,high
2,1,C,2,0.265048,0.911763,0.840081,val1,below
3,0,A,3,0.783205,0.260757,0.278179,val0,veryhigh
4,0,B,4,0.918001,0.766376,0.069914,val0,veryhigh


In [18]:
df.sample()

,binarne,key,data,losowe,losowe2,losowe3,binarne_str,losoweDysk1
5,0,C,5,0.827355,0.261531,0.63311,val0,veryhigh


In [19]:
df.sample(3)

,binarne,key,data,losowe,losowe2,losowe3,binarne_str,losoweDysk1
5,0,C,5,0.827355,0.261531,0.633110,val0,veryhigh
1,1,B,1,0.588542,0.260480,0.386006,val1,high
2,1,C,2,0.265048,0.911763,0.840081,val1,below


In [21]:
df[['losowe','losowe2','losowe3']]

,losowe,losowe2,losowe3
0,0.916874,0.727951,0.122291
1,0.588542,0.260480,0.386006
2,0.265048,0.911763,0.840081
3,0.783205,0.260757,0.278179
4,0.918001,0.766376,0.069914
5,0.827355,0.261531,0.633110


Operacja wykonana w kolumnach (w tym wypadku wybieramy kolumny na których chcemy wykonać operację za pomocą listy)

In [22]:
df[['losowe','losowe2','losowe3']].sum()

losowe     4.299025
losowe2    3.188859
losowe3    2.329581
dtype: float64

Tym razem chcemy wykonać tą operację na wierszach

In [23]:
df[['losowe','losowe2','losowe3']].sum(axis=1)

0    1.767116
1    1.235028
2    2.016892
3    1.322141
4    1.754291
5    1.721996
dtype: float64

### Brakujące dane
Wiecie już jak uzupełniać brakujące dane w kolumnie. Jednak istnieje kilka przypadków, gdzie trzeba trochę się nagimnastykować aby uzyskać porządany efekt.

In [24]:
df.loc[3,"losoweDysk1"] = np.NaN
print(df.dtypes)
df

binarne           int64
key              object
data              int64
losowe          float64
losowe2         float64
losowe3         float64
binarne_str      object
losoweDysk1    category
dtype: object


,binarne,key,data,losowe,losowe2,losowe3,binarne_str,losoweDysk1
0,1,A,0,0.916874,0.727951,0.122291,val1,veryhigh
1,1,B,1,0.588542,0.260480,0.386006,val1,high
2,1,C,2,0.265048,0.911763,0.840081,val1,below
3,0,A,3,0.783205,0.260757,0.278179,val0,NaN
4,0,B,4,0.918001,0.766376,0.069914,val0,veryhigh
5,0,C,5,0.827355,0.261531,0.633110,val0,veryhigh


In [25]:
df["losoweDysk1"].fillna('high')


0    veryhigh
1        high
2       below
3        high
4    veryhigh
5    veryhigh
Name: losoweDysk1, dtype: category
Categories (3, object): ['below' < 'high' < 'veryhigh']

In [26]:
df

,binarne,key,data,losowe,losowe2,losowe3,binarne_str,losoweDysk1
0,1,A,0,0.916874,0.727951,0.122291,val1,veryhigh
1,1,B,1,0.588542,0.260480,0.386006,val1,high
2,1,C,2,0.265048,0.911763,0.840081,val1,below
3,0,A,3,0.783205,0.260757,0.278179,val0,NaN
4,0,B,4,0.918001,0.766376,0.069914,val0,veryhigh
5,0,C,5,0.827355,0.261531,0.633110,val0,veryhigh


Czasem jednak chcemy zastąpić brakujące dane inną etykietą. Dlatego trzeba dodać tą etykietę do zbioru możliwych kategorii

In [29]:
#df["losoweDysk1"].fillna('missing')
df["losoweDysk1"].cat.add_categories(['missing']).fillna('missing')

0    veryhigh
1        high
2       below
3     missing
4    veryhigh
5    veryhigh
Name: losoweDysk1, dtype: category
Categories (4, object): ['below' < 'high' < 'veryhigh' < 'missing']

Widzimy, że kategoria jest uporządkowana a przy dodaniu wartości missing to uporządkowanie może nie mieć sensu. Dlatego możemy skorzystać z metody `as_unordered()`

Istnieje też bliźniacza metoda `as_ordered()`

In [30]:
df["losoweDysk1"].cat.add_categories(['missing']).fillna('missing').cat.as_unordered()

0    veryhigh
1        high
2       below
3     missing
4    veryhigh
5    veryhigh
Name: losoweDysk1, dtype: category
Categories (4, object): ['below', 'high', 'veryhigh', 'missing']

Powyższe operacje wykonywaliśmy w ten sposób, że można uzyskaną kolumnę przypisać do nowej kolumny bądź zastąpić już istniejącą. Jednak bez wskazania co ma być zastąpione pandas pozostawi ramkę bez zmian.

In [31]:
df

,binarne,key,data,losowe,losowe2,losowe3,binarne_str,losoweDysk1
0,1,A,0,0.916874,0.727951,0.122291,val1,veryhigh
1,1,B,1,0.588542,0.260480,0.386006,val1,high
2,1,C,2,0.265048,0.911763,0.840081,val1,below
3,0,A,3,0.783205,0.260757,0.278179,val0,NaN
4,0,B,4,0.918001,0.766376,0.069914,val0,veryhigh
5,0,C,5,0.827355,0.261531,0.633110,val0,veryhigh


Wiele metod w pandas operujących na kolumnach bądź wierszach pozwala na operację "w miejscu" bez potrzeby przypisywania kolumny

In [32]:
df["losoweDysk1"].fillna('high', inplace=True)
df

,binarne,key,data,losowe,losowe2,losowe3,binarne_str,losoweDysk1
0,1,A,0,0.916874,0.727951,0.122291,val1,veryhigh
1,1,B,1,0.588542,0.260480,0.386006,val1,high
2,1,C,2,0.265048,0.911763,0.840081,val1,below
3,0,A,3,0.783205,0.260757,0.278179,val0,high
4,0,B,4,0.918001,0.766376,0.069914,val0,veryhigh
5,0,C,5,0.827355,0.261531,0.633110,val0,veryhigh


to samo tyczy się operacji na całych ramkach

In [33]:
df.drop("binarne", axis=1)

,key,data,losowe,losowe2,losowe3,binarne_str,losoweDysk1
0,A,0,0.916874,0.727951,0.122291,val1,veryhigh
1,B,1,0.588542,0.260480,0.386006,val1,high
2,C,2,0.265048,0.911763,0.840081,val1,below
3,A,3,0.783205,0.260757,0.278179,val0,high
4,B,4,0.918001,0.766376,0.069914,val0,veryhigh
5,C,5,0.827355,0.261531,0.633110,val0,veryhigh


In [34]:
df

,binarne,key,data,losowe,losowe2,losowe3,binarne_str,losoweDysk1
0,1,A,0,0.916874,0.727951,0.122291,val1,veryhigh
1,1,B,1,0.588542,0.260480,0.386006,val1,high
2,1,C,2,0.265048,0.911763,0.840081,val1,below
3,0,A,3,0.783205,0.260757,0.278179,val0,high
4,0,B,4,0.918001,0.766376,0.069914,val0,veryhigh
5,0,C,5,0.827355,0.261531,0.633110,val0,veryhigh


In [35]:
df.drop("binarne", axis=1, inplace=True)

In [36]:
df

,key,data,losowe,losowe2,losowe3,binarne_str,losoweDysk1
0,A,0,0.916874,0.727951,0.122291,val1,veryhigh
1,B,1,0.588542,0.260480,0.386006,val1,high
2,C,2,0.265048,0.911763,0.840081,val1,below
3,A,3,0.783205,0.260757,0.278179,val0,high
4,B,4,0.918001,0.766376,0.069914,val0,veryhigh
5,C,5,0.827355,0.261531,0.633110,val0,veryhigh


Grupowanie i agregacja

In [39]:
df[['key','losowe','losowe2','losowe3']].groupby("key").aggregate("mean")

,losowe,losowe2,losowe3
key,,,
A,0.850039,0.494354,0.200235
B,0.753271,0.513428,0.227960
C,0.546201,0.586647,0.736595


In [40]:
df[['key','losowe','losowe2','losowe3']].groupby("key").agg({"losowe":["sum","min","max"]})

losowe                    
          sum       min       max
key                              
A    1.700079  0.783205  0.916874
B    1.506543  0.588542  0.918001
C    1.092403  0.265048  0.827355

In [41]:
df.groupby("key").agg({"losowe":["sum","min","max"], "losowe2":["mean","var"]})

losowe                       losowe2          
          sum       min       max      mean       var
key                                                  
A    1.700079  0.783205  0.916874  0.494354  0.109136
B    1.506543  0.588542  0.918001  0.513428  0.127965
C    1.092403  0.265048  0.827355  0.586647  0.211401

### Rzutowanie wyniku na różną reprezentację

In [42]:
df.groupby("key").agg({"losowe":["sum","min","max"], "losowe2":["mean","var"]}).to_dict()

{('losowe', 'sum'): {'A': 1.700078961034706,
  'B': 1.5065429761826934,
  'C': 1.0924027631368878},
 ('losowe', 'min'): {'A': 0.7832053795417385,
  'B': 0.5885419136419221,
  'C': 0.26504775117246204},
 ('losowe', 'max'): {'A': 0.9168735814929675,
  'B': 0.9180010625407714,
  'C': 0.8273550119644258},
 ('losowe2', 'mean'): {'A': 0.4943540189094261,
  'B': 0.5134282237641317,
  'C': 0.5866472677598009},
 ('losowe2', 'var'): {'A': 0.10913554977633626,
  'B': 0.12796517846703895,
  'C': 0.2114009980943989}}

In [43]:
df.groupby("key").agg({"losowe":["sum","min","max"], "losowe2":["mean","var"]}).to_latex()

'\\begin{tabular}{lrrrrr}\n\\toprule\n & \\multicolumn{3}{r}{losowe} & \\multicolumn{2}{r}{losowe2} \\\\\n & sum & min & max & mean & var \\\\\nkey &  &  &  &  &  \\\\\n\\midrule\nA & 1.700079 & 0.783205 & 0.916874 & 0.494354 & 0.109136 \\\\\nB & 1.506543 & 0.588542 & 0.918001 & 0.513428 & 0.127965 \\\\\nC & 1.092403 & 0.265048 & 0.827355 & 0.586647 & 0.211401 \\\\\n\\bottomrule\n\\end{tabular}\n'

In [44]:
df.groupby("key").agg({"losowe":["sum","min","max"], "losowe2":["mean","var"]}).to_records()

rec.array([('A', 1.70007896, 0.78320538, 0.91687358, 0.49435402, 0.10913555),
           ('B', 1.50654298, 0.58854191, 0.91800106, 0.51342822, 0.12796518),
           ('C', 1.09240276, 0.26504775, 0.82735501, 0.58664727, 0.211401  )],
          dtype=[('key', 'O'), ("('losowe', 'sum')", '<f8'), ("('losowe', 'min')", '<f8'), ("('losowe', 'max')", '<f8'), ("('losowe2', 'mean')", '<f8'), ("('losowe2', 'var')", '<f8')])

In [45]:
df.groupby("key").agg({"losowe":["sum","min","max"], "losowe2":["mean","var"]}).to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr>\n      <th></th>\n      <th colspan="3" halign="left">losowe</th>\n      <th colspan="2" halign="left">losowe2</th>\n    </tr>\n    <tr>\n      <th></th>\n      <th>sum</th>\n      <th>min</th>\n      <th>max</th>\n      <th>mean</th>\n      <th>var</th>\n    </tr>\n    <tr>\n      <th>key</th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>A</th>\n      <td>1.700079</td>\n      <td>0.783205</td>\n      <td>0.916874</td>\n      <td>0.494354</td>\n      <td>0.109136</td>\n    </tr>\n    <tr>\n      <th>B</th>\n      <td>1.506543</td>\n      <td>0.588542</td>\n      <td>0.918001</td>\n      <td>0.513428</td>\n      <td>0.127965</td>\n    </tr>\n    <tr>\n      <th>C</th>\n      <td>1.092403</td>\n      <td>0.265048</td>\n      <td>0.827355</td>\n      <td>0.586647</td>\n      <td>0.211401</td>\n    </tr>\n  </tbody>\n</table>'

### Łączenie tabel

In [46]:
df2 = pd.DataFrame(
    {
        'key': ['A', 'B', 'C'],
        'val': rng.rand(3) 
    },
    columns=['key', 'val']
)
df2

,key,val
0,A,0.584766
1,B,0.581232
2,C,0.677205


Łączenie po indeksie. (muszą być ustawione indeksy!)

In [49]:
#df.join(df2)
#df.set_index('key')
df.set_index('key').join(df2.set_index('key'))

,data,losowe,losowe2,losowe3,binarne_str,losoweDysk1,val
key,,,,,,,
A,0,0.916874,0.727951,0.122291,val1,veryhigh,0.584766
B,1,0.588542,0.260480,0.386006,val1,high,0.581232
C,2,0.265048,0.911763,0.840081,val1,below,0.677205
A,3,0.783205,0.260757,0.278179,val0,high,0.584766
B,4,0.918001,0.766376,0.069914,val0,veryhigh,0.581232
C,5,0.827355,0.261531,0.633110,val0,veryhigh,0.677205


Łączenie bez indeksów możemy wykonać za pomocą `merge`

```Python
DataFrame.merge(right, 
                how='inner', 
                on=None, 
                left_on=None,
                right_on=None, 
                left_index=False, 
                right_index=False, 
                sort=False, 
                suffixes=('_x', '_y'), #gdy powtarzają się nazwy kolumn
                copy=True, 
                indicator=False, 
                validate=None)```

In [50]:
df.merge(df2, on='key')

,key,data,losowe,losowe2,losowe3,binarne_str,losoweDysk1,val
0,A,0,0.916874,0.727951,0.122291,val1,veryhigh,0.584766
1,B,1,0.588542,0.260480,0.386006,val1,high,0.581232
2,C,2,0.265048,0.911763,0.840081,val1,below,0.677205
3,A,3,0.783205,0.260757,0.278179,val0,high,0.584766
4,B,4,0.918001,0.766376,0.069914,val0,veryhigh,0.581232
5,C,5,0.827355,0.261531,0.633110,val0,veryhigh,0.677205


In [51]:
df

,key,data,losowe,losowe2,losowe3,binarne_str,losoweDysk1
0,A,0,0.916874,0.727951,0.122291,val1,veryhigh
1,B,1,0.588542,0.260480,0.386006,val1,high
2,C,2,0.265048,0.911763,0.840081,val1,below
3,A,3,0.783205,0.260757,0.278179,val0,high
4,B,4,0.918001,0.766376,0.069914,val0,veryhigh
5,C,5,0.827355,0.261531,0.633110,val0,veryhigh


### Pivotowanie

```Python
DataFrame.pivot(index=None, 
                columns=None, 
                values=None)
```

In [52]:
df.pivot(index="key",columns='binarne_str',values='losowe')

binarne_str,val0,val1
key,,
A,0.783205,0.916874
B,0.918001,0.588542
C,0.827355,0.265048


Alternatywnie można użyć metody pozwalającej na podanie więcej parametrów, m.in. agregację dzięki czemu wartości dla poszczególnych kombinacji może być więcej a nie potrzeba pivotowania poprzedzać grupowaniem i agregacją.

```Python
DataFrame.pivot_table(index=None, 
                columns=None, 
                values=None,
                aggfunc='mean',
                fill_value=None,
                #...
                )
```

In [53]:
df.pivot_table(index="key",columns='binarne_str',values='losowe', aggfunc="min")

binarne_str,val0,val1
key,,
A,0.783205,0.916874
B,0.918001,0.588542
C,0.827355,0.265048


In [54]:
arrres = df.pivot(index="key",columns='binarne_str',values='losowe').to_records()
arrres

rec.array([('A', 0.78320538, 0.91687358), ('B', 0.91800106, 0.58854191),
           ('C', 0.82735501, 0.26504775)],
          dtype=[('key', 'O'), ('val0', '<f8'), ('val1', '<f8')])

In [55]:
pd.DataFrame(arrres)

,key,val0,val1
0,A,0.783205,0.916874
1,B,0.918001,0.588542
2,C,0.827355,0.265048


In [56]:
df[['losowe','losowe2','losowe3']].corr()


,losowe,losowe2,losowe3
losowe,1.000000,-0.257678,-0.798480
losowe2,-0.257678,1.000000,0.020087
losowe3,-0.798480,0.020087,1.000000


In [57]:
df.loc[:,'losowe']


0    0.916874
1    0.588542
2    0.265048
3    0.783205
4    0.918001
5    0.827355
Name: losowe, dtype: float64

Łączenie wierszy tabel w jedną tabelę

In [58]:
df2 = pd.concat([df,df], ignore_index=True)
df2

,key,data,losowe,losowe2,losowe3,binarne_str,losoweDysk1
0,A,0,0.916874,0.727951,0.122291,val1,veryhigh
1,B,1,0.588542,0.260480,0.386006,val1,high
2,C,2,0.265048,0.911763,0.840081,val1,below
3,A,3,0.783205,0.260757,0.278179,val0,high
4,B,4,0.918001,0.766376,0.069914,val0,veryhigh
5,C,5,0.827355,0.261531,0.633110,val0,veryhigh
6,A,0,0.916874,0.727951,0.122291,val1,veryhigh
7,B,1,0.588542,0.260480,0.386006,val1,high
8,C,2,0.265048,0.911763,0.840081,val1,below
9,A,3,0.783205,0.260757,0.278179,val0,high


Usuwanie duplikatów

In [59]:

df2.drop_duplicates(subset=["key", "binarne_str"])

,key,data,losowe,losowe2,losowe3,binarne_str,losoweDysk1
0,A,0,0.916874,0.727951,0.122291,val1,veryhigh
1,B,1,0.588542,0.260480,0.386006,val1,high
2,C,2,0.265048,0.911763,0.840081,val1,below
3,A,3,0.783205,0.260757,0.278179,val0,high
4,B,4,0.918001,0.766376,0.069914,val0,veryhigh
5,C,5,0.827355,0.261531,0.633110,val0,veryhigh


Bibliografia:
https://pandas.pydata.org/docs/index.html



## Operacje na ciągach znaków

In [60]:
df = pd.DataFrame(
    {
        'imiona': ["Pan Andrzej Macierzowy", "Pani Gertruda Losowa", "Pani Antonina Wektorowa", "Pan Alojzy Kod", "Pan Antoni Data", "Pani Anastazja Frejm"],
        'wagon': [' A', ' B ', '  C', ' A', '  B', 'C  '],
        'miejsce': [34,12,23,8,21,23],
        'dodatki': ["wars","kawka",np.NaN,"kawka","SNAK","wars"],
        'bagaz': ["maly","Duzy","maly","duzy","brak","duzy"]
    },
    columns=['imiona', 'wagon','miejsce','dodatki','bagaz']
)
df

,imiona,wagon,miejsce,dodatki,bagaz
0,Pan Andrzej Macierzowy,A,34,wars,maly
1,Pani Gertruda Losowa,B,12,kawka,Duzy
2,Pani Antonina Wektorowa,C,23,NaN,maly
3,Pan Alojzy Kod,A,8,kawka,duzy
4,Pan Antoni Data,B,21,SNAK,brak
5,Pani Anastazja Frejm,C,23,wars,duzy


In [61]:
df.columns

Index(['imiona', 'wagon', 'miejsce', 'dodatki', 'bagaz'], dtype='object')

In [62]:
df.columns.str.upper()

Index(['IMIONA', 'WAGON', 'MIEJSCE', 'DODATKI', 'BAGAZ'], dtype='object')

In [63]:
df.columns = df.columns.str.upper()

In [64]:
df

,IMIONA,WAGON,MIEJSCE,DODATKI,BAGAZ
0,Pan Andrzej Macierzowy,A,34,wars,maly
1,Pani Gertruda Losowa,B,12,kawka,Duzy
2,Pani Antonina Wektorowa,C,23,NaN,maly
3,Pan Alojzy Kod,A,8,kawka,duzy
4,Pan Antoni Data,B,21,SNAK,brak
5,Pani Anastazja Frejm,C,23,wars,duzy


In [65]:
df.WAGON == "B"

0    False
1    False
2    False
3    False
4    False
5    False
Name: WAGON, dtype: bool

In [66]:
df.WAGON

0      A
1     B 
2      C
3      A
4      B
5    C  
Name: WAGON, dtype: object

In [67]:
"\"" + df.WAGON + "\""

0     " A"
1    " B "
2    "  C"
3     " A"
4    "  B"
5    "C  "
Name: WAGON, dtype: object

In [68]:
"\"" + df.WAGON.str.strip() + "\""

0    "A"
1    "B"
2    "C"
3    "A"
4    "B"
5    "C"
Name: WAGON, dtype: object

In [69]:
df.WAGON = df.WAGON.str.strip()

In [70]:
df

,IMIONA,WAGON,MIEJSCE,DODATKI,BAGAZ
0,Pan Andrzej Macierzowy,A,34,wars,maly
1,Pani Gertruda Losowa,B,12,kawka,Duzy
2,Pani Antonina Wektorowa,C,23,NaN,maly
3,Pan Alojzy Kod,A,8,kawka,duzy
4,Pan Antoni Data,B,21,SNAK,brak
5,Pani Anastazja Frejm,C,23,wars,duzy


In [71]:
df.IMIONA.str.split(" ")

0     [Pan, Andrzej, Macierzowy]
1       [Pani, Gertruda, Losowa]
2    [Pani, Antonina, Wektorowa]
3             [Pan, Alojzy, Kod]
4            [Pan, Antoni, Data]
5       [Pani, Anastazja, Frejm]
Name: IMIONA, dtype: object

In [72]:
df.IMIONA.str.split(" ")[0]

['Pan', 'Andrzej', 'Macierzowy']

In [73]:
df.IMIONA.str.split(" ").str[0]

0     Pan
1    Pani
2    Pani
3     Pan
4     Pan
5    Pani
Name: IMIONA, dtype: object

In [74]:
df["TYTUL"] = df.IMIONA.str.split(" ").str[0]
df["IMIE"] = df.IMIONA.str.split(" ").str[1]
df["NAZWISKO"] = df.IMIONA.str.split(" ").str[-1]

In [75]:
df

,IMIONA,WAGON,MIEJSCE,DODATKI,BAGAZ,TYTUL,IMIE,NAZWISKO
0,Pan Andrzej Macierzowy,A,34,wars,maly,Pan,Andrzej,Macierzowy
1,Pani Gertruda Losowa,B,12,kawka,Duzy,Pani,Gertruda,Losowa
2,Pani Antonina Wektorowa,C,23,NaN,maly,Pani,Antonina,Wektorowa
3,Pan Alojzy Kod,A,8,kawka,duzy,Pan,Alojzy,Kod
4,Pan Antoni Data,B,21,SNAK,brak,Pan,Antoni,Data
5,Pani Anastazja Frejm,C,23,wars,duzy,Pani,Anastazja,Frejm


In [76]:
ddf = df.IMIONA.str.split(" ", expand=True) #od razu robi ramkę
ddf.columns = ["A","B", "CC"]
ddf

,A,B,CC
0,Pan,Andrzej,Macierzowy
1,Pani,Gertruda,Losowa
2,Pani,Antonina,Wektorowa
3,Pan,Alojzy,Kod
4,Pan,Antoni,Data
5,Pani,Anastazja,Frejm


In [77]:
df.TYTUL.str.len()

0    3
1    4
2    4
3    3
4    3
5    4
Name: TYTUL, dtype: int64

In [78]:
df.TYTUL.replace("Pani", "Panią")

0      Pan
1    Panią
2    Panią
3      Pan
4      Pan
5    Panią
Name: TYTUL, dtype: object

In [79]:
df["ODMIANA"] = df.TYTUL.replace("Pani", "Panią") # dopasowuje regexp z pierwszego argumentu

In [80]:
df.ODMIANA=="Pan"

0     True
1    False
2    False
3     True
4     True
5    False
Name: ODMIANA, dtype: bool

In [81]:
df.loc[df.ODMIANA=="Pan","ODMIANA"] = "Panem"
df

,IMIONA,WAGON,MIEJSCE,DODATKI,BAGAZ,TYTUL,IMIE,NAZWISKO,ODMIANA
0,Pan Andrzej Macierzowy,A,34,wars,maly,Pan,Andrzej,Macierzowy,Panem
1,Pani Gertruda Losowa,B,12,kawka,Duzy,Pani,Gertruda,Losowa,Panią
2,Pani Antonina Wektorowa,C,23,NaN,maly,Pani,Antonina,Wektorowa,Panią
3,Pan Alojzy Kod,A,8,kawka,duzy,Pan,Alojzy,Kod,Panem
4,Pan Antoni Data,B,21,SNAK,brak,Pan,Antoni,Data,Panem
5,Pani Anastazja Frejm,C,23,wars,duzy,Pani,Anastazja,Frejm,Panią


In [82]:
df.WAGON.str.cat(sep=", ") # łączenie ciągów znaków kolumny

'A, B, C, A, B, C'

In [83]:
df.WAGON.str.cat(df.MIEJSCE.astype("str")) # łączenie łańcuchów w różnych kolumnach

0    A34
1    B12
2    C23
3     A8
4    B21
5    C23
Name: WAGON, dtype: object

In [84]:
df["SIEDZENIE"] = df.WAGON.str.cat(df.MIEJSCE.astype("str"))

In [85]:
df

,IMIONA,WAGON,MIEJSCE,DODATKI,BAGAZ,TYTUL,IMIE,NAZWISKO,ODMIANA,SIEDZENIE
0,Pan Andrzej Macierzowy,A,34,wars,maly,Pan,Andrzej,Macierzowy,Panem,A34
1,Pani Gertruda Losowa,B,12,kawka,Duzy,Pani,Gertruda,Losowa,Panią,B12
2,Pani Antonina Wektorowa,C,23,NaN,maly,Pani,Antonina,Wektorowa,Panią,C23
3,Pan Alojzy Kod,A,8,kawka,duzy,Pan,Alojzy,Kod,Panem,A8
4,Pan Antoni Data,B,21,SNAK,brak,Pan,Antoni,Data,Panem,B21
5,Pani Anastazja Frejm,C,23,wars,duzy,Pani,Anastazja,Frejm,Panią,C23


In [86]:
df.BAGAZ = df.BAGAZ.str.lower()
df

,IMIONA,WAGON,MIEJSCE,DODATKI,BAGAZ,TYTUL,IMIE,NAZWISKO,ODMIANA,SIEDZENIE
0,Pan Andrzej Macierzowy,A,34,wars,maly,Pan,Andrzej,Macierzowy,Panem,A34
1,Pani Gertruda Losowa,B,12,kawka,duzy,Pani,Gertruda,Losowa,Panią,B12
2,Pani Antonina Wektorowa,C,23,NaN,maly,Pani,Antonina,Wektorowa,Panią,C23
3,Pan Alojzy Kod,A,8,kawka,duzy,Pan,Alojzy,Kod,Panem,A8
4,Pan Antoni Data,B,21,SNAK,brak,Pan,Antoni,Data,Panem,B21
5,Pani Anastazja Frejm,C,23,wars,duzy,Pani,Anastazja,Frejm,Panią,C23


In [87]:
df.WAGON+df.DODATKI

0     Awars
1    Bkawka
2       NaN
3    Akawka
4     BSNAK
5     Cwars
dtype: object

In [88]:
df.WAGON.str.cat(df.DODATKI)

0     Awars
1    Bkawka
2       NaN
3    Akawka
4     BSNAK
5     Cwars
Name: WAGON, dtype: object

In [89]:
df.WAGON.str.cat(df.DODATKI, na_rep="-")

0     Awars
1    Bkawka
2        C-
3    Akawka
4     BSNAK
5     Cwars
Name: WAGON, dtype: object

In [90]:
df

,IMIONA,WAGON,MIEJSCE,DODATKI,BAGAZ,TYTUL,IMIE,NAZWISKO,ODMIANA,SIEDZENIE
0,Pan Andrzej Macierzowy,A,34,wars,maly,Pan,Andrzej,Macierzowy,Panem,A34
1,Pani Gertruda Losowa,B,12,kawka,duzy,Pani,Gertruda,Losowa,Panią,B12
2,Pani Antonina Wektorowa,C,23,NaN,maly,Pani,Antonina,Wektorowa,Panią,C23
3,Pan Alojzy Kod,A,8,kawka,duzy,Pan,Alojzy,Kod,Panem,A8
4,Pan Antoni Data,B,21,SNAK,brak,Pan,Antoni,Data,Panem,B21
5,Pani Anastazja Frejm,C,23,wars,duzy,Pani,Anastazja,Frejm,Panią,C23


In [91]:
df.IMIONA.str.startswith("Pani")

0    False
1     True
2     True
3    False
4    False
5     True
Name: IMIONA, dtype: bool

In [92]:
siedzeniaMap = {0:"LO", 1:"LK", 2:"PK", 3:"PO"} #lewe okno, lewe korytarz, prawe korytarz, prawe okno
def rzadUsytuowanie(x):
    return str(x//4) + siedzeniaMap[x%4]

In [93]:
df.MIEJSCE.apply(rzadUsytuowanie)

0    8PK
1    3LO
2    5PO
3    2LO
4    5LK
5    5PO
Name: MIEJSCE, dtype: object

Metody "apply" są bardzo użyteczne i wykorzystują przetwarzanie wektorowe, które jest dobrze optymalizowane i działa szybciej niż iterowanie pętlą po kolejnych wierszach. Takie operacje można dodatkowo zrównoleglać. O tym na następnym wykładzie.